# Aula extra - Como analisar os dados de rentabilidade?

### Desafio:

* Construir um código que faça um ranking dos melhores fundos em três períodos de rentabilidade diferentes. Iremos selecionar os 10 melhores posicionados no quesito rentabilidade no curto e no longo prazo.

* Essa aula será mais curta. 

### Passo a passo:

   **Passo 1** - Puxar as tabelas igual na aula 2.
   
   **Passo 2** - Escolher quais períodos de rentabilidade iremos analisar.
   
   **Passo 3** - Retirar os dados faltantes. Fundos que não possuem os períodos necessários, ficarão de fora. 

   **Passo 4** - Transformar as rentabilidades em números decimais, ao invés de string. 
   
   **Passo 5** - Juntar as tabelas, igual na aula 2. 
   
   **Passo 6** - Filtrar os ETFs alavancados. 
   
   **Passo 7** - Fazer os rankings de rentabilidade por período e soma-los. 
   
   **Passo 8** - Ordenar do menor ranking pro maior, encontrando os "melhores" ETFs.

# Passo 1: Puxar as tabelas igual na aula 2.

In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

In [15]:
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))

url = "https://www.etf.com/etfanalytics/etf-finder"

driver.get(url)

time.sleep(5)

botao_100 = driver.find_element("xpath", '''html/body/div[5]/section/div/div[3]/section/div
                                                /div/div/div/div[2]/
                                section[2]/div[2]/section[2]/div[1]/div/div[4]/button/label/span''')

driver.execute_script("arguments[0].click();", botao_100)

numero_paginas = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/
section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[2]/div/label[2]''')

numero_paginas = numero_paginas.text.replace("of ", "")

numero_paginas = int(numero_paginas)

lista_de_tabela_por_pagina = []

for pagina in range(0, numero_paginas):
    
    tabela = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/section/div/
                                    div/div/div/div[2]/section[2]/div[2]/div/table''')

    html_tabela = tabela.get_attribute("outerHTML")

    tabela_final = pd.read_html(html_tabela)[0]
    
    lista_de_tabela_por_pagina.append(tabela_final)
    
    botao_avancar_pagina = driver.find_element("xpath", '//*[@id="nextPage"]')
    
    driver.execute_script("arguments[0].click();", botao_avancar_pagina)
    

base_de_dados_completa = pd.concat(lista_de_tabela_por_pagina)

botao_aba = driver.find_element("xpath", ''' /html/body/div[5]/section/div/div[3]/section/div/div/div/div/
div[2]/section[2]/div[2]/ul/li[2]/span''')

driver.execute_script("arguments[0].click();", botao_aba)

#voltando a 1 página

for pagina in range(0, numero_paginas):
    
    botao_voltar_pagina = driver.find_element("xpath", '//*[@id="previousPage"]')
    
    driver.execute_script("arguments[0].click();", botao_voltar_pagina)
    
    
lista_de_tabela_por_pagina = []

for pagina in range(0, numero_paginas):
    
    tabela = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/section/div/
                                    div/div/div/div[2]/section[2]/div[2]/div/table''')

    html_tabela = tabela.get_attribute("outerHTML")

    tabela_final = pd.read_html(html_tabela)[0]
    
    lista_de_tabela_por_pagina.append(tabela_final)
    
    botao_avancar_pagina = driver.find_element("xpath", '//*[@id="nextPage"]')
    
    driver.execute_script("arguments[0].click();", botao_avancar_pagina)
    

base_de_dados_performance = pd.concat(lista_de_tabela_por_pagina)

driver.quit()

In [16]:
base_de_dados_completa = base_de_dados_completa.set_index('Ticker')
base_de_dados_performance = base_de_dados_performance.set_index("Ticker")

In [17]:
base_de_dados_performance

,Name,1 Month,3 Month,YTD,1 Year,3 Years,5 Years,10 Years,As Of Date
Ticker,,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,-4.08%,1.77%,4.14%,-3.50%,12.01%,9.74%,11.99%,03/07/23
IVV,iShares Core S&P 500 ETF,-4.07%,1.77%,4.14%,-3.48%,12.03%,9.77%,12.04%,03/07/23
VTI,Vanguard Total Stock Market ETF,-4.12%,2.33%,4.75%,-3.64%,11.69%,9.19%,11.63%,03/07/23
VOO,Vanguard S&P 500 ETF,-4.07%,1.77%,4.17%,-3.51%,12.02%,9.76%,12.04%,03/07/23
QQQ,Invesco QQQ Trust,-4.37%,5.89%,11.29%,-8.11%,13.24%,12.70%,16.83%,03/07/23
...,...,...,...,...,...,...,...,...,...
CVLC,Calvert US Large-Cap Core Responsible Index ETF,-4.29%,--,--,--,--,--,--,03/07/23
CVMC,Calvert US Mid-Cap Core Responsible Index ETF,-4.24%,--,--,--,--,--,--,03/07/23
CVIE,Calvert International Responsible Index ETF,-2.85%,--,--,--,--,--,--,03/07/23


# Passo 2: Escolher quais períodos de rentabilidade iremos analisar.

No nosso caso, iremos pegar as janelas de 3, 5 e 10 anos. 

In [18]:
base_de_dados_performance = base_de_dados_performance[['3 Years', '5 Years', '10 Years']]

base_de_dados_performance

,3 Years,5 Years,10 Years
Ticker,,,
SPY,12.01%,9.74%,11.99%
IVV,12.03%,9.77%,12.04%
VTI,11.69%,9.19%,11.63%
VOO,12.02%,9.76%,12.04%
QQQ,13.24%,12.70%,16.83%
...,...,...,...
CVLC,--,--,--
CVMC,--,--,--
CVIE,--,--,--


# Passo 3: Retirar os dados faltantes. Fundos que não possuem os períodos necessários, ficarão de fora. 

In [19]:
base_de_dados_performance = base_de_dados_performance.replace("--", pd.NA)

base_de_dados_performance

,3 Years,5 Years,10 Years
Ticker,,,
SPY,12.01%,9.74%,11.99%
IVV,12.03%,9.77%,12.04%
VTI,11.69%,9.19%,11.63%
VOO,12.02%,9.76%,12.04%
QQQ,13.24%,12.70%,16.83%
...,...,...,...
CVLC,NaN,NaN,NaN
CVMC,NaN,NaN,NaN
CVIE,NaN,NaN,NaN


In [20]:
base_de_dados_performance = base_de_dados_performance.dropna()

base_de_dados_performance

,3 Years,5 Years,10 Years
Ticker,,,
SPY,12.01%,9.74%,11.99%
IVV,12.03%,9.77%,12.04%
VTI,11.69%,9.19%,11.63%
VOO,12.02%,9.76%,12.04%
QQQ,13.24%,12.70%,16.83%
...,...,...,...
SZK,-31.20%,-23.71%,-22.76%
ERUS,-36.21%,-22.94%,-12.71%
LD,5.36%,-3.44%,-1.52%


# Passo 4: Transformar as rentabilidades em números decimais

In [21]:
base_de_dados_performance['3 Years'] = (base_de_dados_performance['3 Years'].
                                        str.rstrip('%').astype(float)/100)

base_de_dados_performance['5 Years'] = (base_de_dados_performance['5 Years'].
                                        str.rstrip('%').astype(float)/100)

base_de_dados_performance['10 Years'] = (base_de_dados_performance['10 Years'].
                                        str.rstrip('%').astype(float)/100)

base_de_dados_performance

C:\Users\rmnf\AppData\Local\Temp\ipykernel_13736\785547042.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_de_dados_performance['3 Years'] = (base_de_dados_performance['3 Years'].
C:\Users\rmnf\AppData\Local\Temp\ipykernel_13736\785547042.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_de_dados_performance['5 Years'] = (base_de_dados_performance['5 Years'].
C:\Users\rmnf\AppData\Local\Temp\ipykernel_13736\785547042.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

,3 Years,5 Years,10 Years
Ticker,,,
SPY,0.1201,0.0974,0.1199
IVV,0.1203,0.0977,0.1204
VTI,0.1169,0.0919,0.1163
VOO,0.1202,0.0976,0.1204
QQQ,0.1324,0.1270,0.1683
...,...,...,...
SZK,-0.3120,-0.2371,-0.2276
ERUS,-0.3621,-0.2294,-0.1271
LD,0.0536,-0.0344,-0.0152


# Passo 5: Juntar as tabelas, igual na aula 2.

In [22]:
base_de_dados_final = base_de_dados_completa.join(base_de_dados_performance, how = "inner")

base_de_dados_final

,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,10 Years
Ticker,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$366.87B,0.1201,0.0974,0.1199
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$303.19B,0.1203,0.0977,0.1204
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$280.93B,0.1169,0.0919,0.1163
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$279.95B,0.1202,0.0976,0.1204
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$159.95B,0.1324,0.1270,0.1683
...,...,...,...,...,...,...,...,...
SZK,ProShares UltraShort Consumer Goods,Inverse Equity: U.S. Consumer Staples,ProShares,0.95%,$1.68M,-0.3120,-0.2371,-0.2276
ERUS,iShares MSCI Russia ETF,Equity: Russia - Total Market,Blackrock,0.08%,$961.13K,-0.3621,-0.2294,-0.1271
LD,iPath Bloomberg Lead Subindex Total Return ETN,Commodities: Industrial Metals Lead,Barclays Capital Inc.,0.70%,$873.29K,0.0536,-0.0344,-0.0152


# Passo 6: Filtrar os ETFs alavancados.

In [23]:
base_de_dados_final = base_de_dados_final[~base_de_dados_final['Segment'].str.contains("Leveraged")]
base_de_dados_final = base_de_dados_final[base_de_dados_final['Segment'].str.contains("China")]

base_de_dados_final

,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,10 Years
Ticker,,,,,,,,
MCHI,iShares MSCI China ETF,Equity: China - Total Market,Blackrock,0.58%,$8.67B,-0.0647,-0.0564,0.0215
FXI,iShares Trust - China Large-Cap ETF,Equity: China - Large Cap,Blackrock,0.74%,$5.75B,-0.0872,-0.0743,-0.0042
GXC,SPDR S&P China ETF,Equity: China - Total Market,State Street Global Advisors,0.59%,$1.27B,-0.0518,-0.0489,0.0314
CQQQ,Invesco China Technology ETF,Equity: China Information Technology,Invesco,0.70%,$1.00B,-0.0658,-0.0700,0.0710
CXSE,WisdomTree China ex-State-Owned Enterprises Fund,Equity: China - Total Market,WisdomTree,0.32%,$793.45M,-0.0572,-0.0426,0.0402
CHIQ,Global X MSCI China Consumer Discretionary ETF,Equity: China Consumer Discretionary,"Mirae Asset Global Investments Co., Ltd.",0.65%,$361.34M,0.0371,0.0203,0.0476
PGJ,Invesco Golden Dragon China ETF,Equity: China - Total Market,Invesco,0.70%,$247.39M,-0.1008,-0.0935,0.0534
YANG,Direxion Daily FTSE China Bear 3X Shares,Inverse Equity: China - Large Cap,Direxion,1.08%,$164.68M,-0.3931,-0.2643,-0.3832
ECNS,iShares MSCI China Small-Cap ETF,Equity: China - Small Cap,Blackrock,0.58%,$78.03M,0.0140,-0.0318,0.0169


# Passo 7: Fazer os rankings de rentabilidade por período e soma-los. 

In [24]:
base_de_dados_final['rank_3_anos'] = base_de_dados_final['3 Years'].rank(ascending = False)
base_de_dados_final['rank_5_anos'] = base_de_dados_final['5 Years'].rank(ascending = False)
base_de_dados_final['rank_10_anos'] = base_de_dados_final['10 Years'].rank(ascending = False)

base_de_dados_final

,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,10 Years,rank_3_anos,rank_5_anos,rank_10_anos
Ticker,,,,,,,,,,,
MCHI,iShares MSCI China ETF,Equity: China - Total Market,Blackrock,0.58%,$8.67B,-0.0647,-0.0564,0.0215,12.0,11.0,11.0
FXI,iShares Trust - China Large-Cap ETF,Equity: China - Large Cap,Blackrock,0.74%,$5.75B,-0.0872,-0.0743,-0.0042,14.0,14.0,15.0
GXC,SPDR S&P China ETF,Equity: China - Total Market,State Street Global Advisors,0.59%,$1.27B,-0.0518,-0.0489,0.0314,9.0,10.0,6.0
CQQQ,Invesco China Technology ETF,Equity: China Information Technology,Invesco,0.70%,$1.00B,-0.0658,-0.0700,0.0710,13.0,13.0,1.0
CXSE,WisdomTree China ex-State-Owned Enterprises Fund,Equity: China - Total Market,WisdomTree,0.32%,$793.45M,-0.0572,-0.0426,0.0402,11.0,8.0,4.0
CHIQ,Global X MSCI China Consumer Discretionary ETF,Equity: China Consumer Discretionary,"Mirae Asset Global Investments Co., Ltd.",0.65%,$361.34M,0.0371,0.0203,0.0476,3.0,2.0,3.0
PGJ,Invesco Golden Dragon China ETF,Equity: China - Total Market,Invesco,0.70%,$247.39M,-0.1008,-0.0935,0.0534,15.0,15.0,2.0
YANG,Direxion Daily FTSE China Bear 3X Shares,Inverse Equity: China - Large Cap,Direxion,1.08%,$164.68M,-0.3931,-0.2643,-0.3832,18.0,18.0,18.0
ECNS,iShares MSCI China Small-Cap ETF,Equity: China - Small Cap,Blackrock,0.58%,$78.03M,0.0140,-0.0318,0.0169,5.0,7.0,13.0


In [25]:
base_de_dados_final['rank_final'] =  (base_de_dados_final['rank_5_anos'] + 
                                      base_de_dados_final['rank_10_anos']) 

base_de_dados_final

,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,10 Years,rank_3_anos,rank_5_anos,rank_10_anos,rank_final
Ticker,,,,,,,,,,,,
MCHI,iShares MSCI China ETF,Equity: China - Total Market,Blackrock,0.58%,$8.67B,-0.0647,-0.0564,0.0215,12.0,11.0,11.0,22.0
FXI,iShares Trust - China Large-Cap ETF,Equity: China - Large Cap,Blackrock,0.74%,$5.75B,-0.0872,-0.0743,-0.0042,14.0,14.0,15.0,29.0
GXC,SPDR S&P China ETF,Equity: China - Total Market,State Street Global Advisors,0.59%,$1.27B,-0.0518,-0.0489,0.0314,9.0,10.0,6.0,16.0
CQQQ,Invesco China Technology ETF,Equity: China Information Technology,Invesco,0.70%,$1.00B,-0.0658,-0.0700,0.0710,13.0,13.0,1.0,14.0
CXSE,WisdomTree China ex-State-Owned Enterprises Fund,Equity: China - Total Market,WisdomTree,0.32%,$793.45M,-0.0572,-0.0426,0.0402,11.0,8.0,4.0,12.0
CHIQ,Global X MSCI China Consumer Discretionary ETF,Equity: China Consumer Discretionary,"Mirae Asset Global Investments Co., Ltd.",0.65%,$361.34M,0.0371,0.0203,0.0476,3.0,2.0,3.0,5.0
PGJ,Invesco Golden Dragon China ETF,Equity: China - Total Market,Invesco,0.70%,$247.39M,-0.1008,-0.0935,0.0534,15.0,15.0,2.0,17.0
YANG,Direxion Daily FTSE China Bear 3X Shares,Inverse Equity: China - Large Cap,Direxion,1.08%,$164.68M,-0.3931,-0.2643,-0.3832,18.0,18.0,18.0,36.0
ECNS,iShares MSCI China Small-Cap ETF,Equity: China - Small Cap,Blackrock,0.58%,$78.03M,0.0140,-0.0318,0.0169,5.0,7.0,13.0,20.0


# Passo 8: Gran finale - Ordenar do menor ranking pro maior, encontrando os "melhores" ETFs.

In [26]:
melhores_etfs_china = base_de_dados_final.sort_values(by = "rank_final")

melhores_etfs_china

,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,10 Years,rank_3_anos,rank_5_anos,rank_10_anos,rank_final
Ticker,,,,,,,,,,,,
CHIQ,Global X MSCI China Consumer Discretionary ETF,Equity: China Consumer Discretionary,"Mirae Asset Global Investments Co., Ltd.",0.65%,$361.34M,0.0371,0.0203,0.0476,3.0,2.0,3.0,5.0
CHIE,Global X MSCI China Energy ETF,Equity: China Energy,"Mirae Asset Global Investments Co., Ltd.",0.66%,$10.81M,0.2325,0.1055,0.0339,1.0,1.0,5.0,6.0
CHIM,Global X MSCI China Materials ETF,Equity: China Materials,"Mirae Asset Global Investments Co., Ltd.",0.65%,$3.68M,0.0718,0.0006,0.0312,2.0,3.0,7.0,10.0
CXSE,WisdomTree China ex-State-Owned Enterprises Fund,Equity: China - Total Market,WisdomTree,0.32%,$793.45M,-0.0572,-0.0426,0.0402,11.0,8.0,4.0,12.0
CHII,Global X MSCI China Industrials ETF,Equity: China Industrials,"Mirae Asset Global Investments Co., Ltd.",0.65%,$6.17M,0.0247,-0.0043,0.0243,4.0,4.0,9.0,13.0
FCA,First Trust China AlphaDEX Fund,Equity: China - Total Market,First Trust,0.80%,$10.35M,0.0106,-0.0215,0.0278,6.0,6.0,8.0,14.0
CQQQ,Invesco China Technology ETF,Equity: China Information Technology,Invesco,0.70%,$1.00B,-0.0658,-0.0700,0.0710,13.0,13.0,1.0,14.0
GXC,SPDR S&P China ETF,Equity: China - Total Market,State Street Global Advisors,0.59%,$1.27B,-0.0518,-0.0489,0.0314,9.0,10.0,6.0,16.0
PGJ,Invesco Golden Dragon China ETF,Equity: China - Total Market,Invesco,0.70%,$247.39M,-0.1008,-0.0935,0.0534,15.0,15.0,2.0,17.0


In [27]:
melhores_etfs_china["3 Years"] = melhores_etfs_china["3 Years"]*100
melhores_etfs_china["5 Years"] = melhores_etfs_china["5 Years"]*100
melhores_etfs_china["10 Years"] = melhores_etfs_china["10 Years"]*100

melhores_etfs_china.head()

,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,10 Years,rank_3_anos,rank_5_anos,rank_10_anos,rank_final
Ticker,,,,,,,,,,,,
CHIQ,Global X MSCI China Consumer Discretionary ETF,Equity: China Consumer Discretionary,"Mirae Asset Global Investments Co., Ltd.",0.65%,$361.34M,3.71,2.03,4.76,3.0,2.0,3.0,5.0
CHIE,Global X MSCI China Energy ETF,Equity: China Energy,"Mirae Asset Global Investments Co., Ltd.",0.66%,$10.81M,23.25,10.55,3.39,1.0,1.0,5.0,6.0
CHIM,Global X MSCI China Materials ETF,Equity: China Materials,"Mirae Asset Global Investments Co., Ltd.",0.65%,$3.68M,7.18,0.06,3.12,2.0,3.0,7.0,10.0
CXSE,WisdomTree China ex-State-Owned Enterprises Fund,Equity: China - Total Market,WisdomTree,0.32%,$793.45M,-5.72,-4.26,4.02,11.0,8.0,4.0,12.0
CHII,Global X MSCI China Industrials ETF,Equity: China Industrials,"Mirae Asset Global Investments Co., Ltd.",0.65%,$6.17M,2.47,-0.43,2.43,4.0,4.0,9.0,13.0
